In [4]:
%pip install dask
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: c:\Users\maxvi\.pyenv\pyenv-win\versions\3.12.7\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/28.0 MB ? eta -:--:--
   ---- ----------------------------------- 2.9/28.0 MB 16.7 MB/s eta 0:00:02
   --------- ------------------------------ 6.6/28.0 MB 16.8 MB/s eta 0:00:02
   -------------- ------------------------- 10.0/28.0 MB 16.3 MB/s eta 0:00:02
   ------------------- -------------------- 13.4/28.0 MB 16.1 MB/s eta 0:00:01
   ---------------------- ----------------- 16.0/28.0 MB 15.2 MB/s eta 0:00:01
   ------------------------- -------------- 17.8/28.0 MB 14.4 MB/s eta 0:00:01
   ----------------------------- ---------- 20.7/28.0 MB 13.9 MB/s eta 0:00:01
   ---------------------------------- ----- 24.1/28.0 MB 14.3 MB/s eta 0:00:01
   ------------------------------------- -- 26.5/28.0 MB 13.9 MB/s eta 0:00:01
   ---------------------------------------  27.8/28.0 MB 13.9 MB/s eta 0:00:01
   ---------------------------------------- 28.0/28.0 MB 13.0 MB/s  0:00:02
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: c:\Users\maxvi\.pyenv\pyenv-win\versions\3.12.7\python.exe -m pip install --upgrade pip


In [1]:
import dask
import dask.dataframe as dd
# When dask uses pandas 2.0.0+ it casts object columns to string automatically ([("i", 0.48)] -> '[("i", 0.48)]')
dask.config.set({"dataframe.convert-string": False})

dataset_path = r"C:\Users\maxvi\Downloads\smadex-challenge-predict-the-revenue\train"
filters = [("datetime", ">=", "2025-10-01-00-00"), ("datetime", "<", "2025-10-13-00-00")]

ddf = dd.read_parquet(
    dataset_path,
    filters=filters
)


In [2]:
# Convertim la Sèrie de Dtypes a un DataFrame i després a llista
dtypes_list = ddf.dtypes.to_frame().reset_index().values.tolist()

# Imprimeix la llista completa. Això és menys probable que es trunqui
for column, dtype in dtypes_list:
    print(f"{column}: {dtype}")

buyer_d1: int32
buyer_d7: int32
buyer_d14: int32
buyer_d28: int32
buy_d7: int64
buy_d14: int64
buy_d28: int64
iap_revenue_d7: float64
iap_revenue_d14: float64
iap_revenue_d28: float64
registration: int32
retention_d1_to_d7: int32
retention_d3_to_d7: int32
retention_d7_to_d14: int32
retention_d1: int32
retention_d3: int32
retentiond7: int32
advertiser_bundle: object
advertiser_category: object
advertiser_subcategory: object
advertiser_bottom_taxonomy_level: object
carrier: object
country: object
region: object
dev_make: object
dev_model: object
dev_os: object
dev_osv: object
hour: object
release_date: object
release_msrp: int64
weekday: int32
avg_act_days: float64
avg_daily_sessions: object
avg_days_ins: float64
avg_duration: object
bcat: object
bcat_bottom_taxonomy: object
bundles_cat: object
bundles_cat_bottom_taxonomy: object
bundles_ins: object
city_hist: object
country_hist: object
cpm: object
cpm_pct_rk: object
ctr: object
ctr_pct_rk: object
dev_language_hist: object
dev_osv_hist:

In [ ]:
ddf_min = ddf[['iap_revenue_d7','buyer_d7','buy_d7','retention_d3_to_d7','datetime','advertiser_category','release_msrp','iap_revenue_usd_bundle','iap_revenue_category','num_buys_bundle','week_since_first_seen']]
ddf_min.head()


,iap_revenue_d7,buyer_d7,retention_d3_to_d7,datetime
0,2.147718,1,1.0,2025-10-01-00-00
1,0.000000,0,0.0,2025-10-01-00-00
2,0.000000,0,1.0,2025-10-01-00-00
3,0.000000,0,NaN,2025-10-01-00-00
4,0.000000,0,0.0,2025-10-01-00-00
